## Integrantes
### Alejandro Orozco 2402036-7727
### Fernando Valencia 2401899-7727
### Carlos Botero 2400879-7727
### Brandon Rivas 2400430-7727

In [ ]:
# Instalación de librerías necesarias
!pip install -U transformers
!pip install torch


In [ ]:
oraciones = [
    "El paciente fue diagnosticado con adenocarcinoma de pulmón estadio IV el 12 de marzo de 2022, con metástasis en cerebro y hueso, y presentó un TNM de T3N2M1.",
    "Se realizó resección quirúrgica del lóbulo superior derecho el 5 de enero, seguida de quimioterapia con carboplatino y pemetrexed durante 4 ciclos cada 21 días.",
    "El paciente con antecedentes familiares de cáncer de pulmón en su padre y antecedentes personales de tabaquismo severo (fumador activo durante 30 años) inició tratamiento con inmunoterapia en marzo.",
    "Se detecta progresión de la enfermedad con aparición de nuevos nódulos pulmonares y derrame pleural bilateral según tomografía de control del 15 de junio.",
    "La paciente presentó evento de hemoptisis severa el 20 de abril, lo que motivó la suspensión temporal de la quimioterapia y la indicación de radioterapia paliativa sobre masa mediastínica.",
    "Tras la cirugía toracoscópica realizada en noviembre de 2021, se administró quimioterapia adyuvante y posteriormente radioterapia sobre la región hilia derecha con dosis de 60 Gy en 30 fracciones.",
    "Paciente exfumador desde hace 5 años, con consumo previo de 20 paquetes/año, presenta adenocarcinoma de pulmón con mutación en EGFR, tratado con osimertinib desde septiembre.",
    "El informe PET/CT del 3 de mayo muestra captación hipermetabólica en ganglios mediastínicos y lesión pulmonar de 2.5 cm, compatible con progresión local.",
    "A pesar de los 6 ciclos de quimioterapia con cisplatino y etopósido administrados cada 21 días, se observa avance a estadio IV con metástasis hepáticas.",
    "Actualmente, el paciente se encuentra en tratamiento con radioterapia estereotáctica iniciada hace dos semanas sobre metástasis cerebral única, complementado con control sistémico con pembrolizumab."
]


## 📄 Descripción breve del código NER

### 🔧 Función principal
Realiza reconocimiento de entidades nombradas (NER) en textos relacionados con pulmonología usando el modelo `FernandoValencia/xlm-roberta-base-finetuned-ner-pulmon`.

### 🚀 Pasos del código
1. **Carga del modelo y tokenizador**:
   - Usa `AutoTokenizer` y `pipeline` de Hugging Face para NER.
   - Configura `aggregation_strategy="simple"` para agrupar tokens en entidades.

2. **Función `merge_spans()`**:
   - Fusiona entidades consecutivas del mismo tipo que están adyacentes en el texto.
   - Evita que entidades fragmentadas se presenten por separado.

3. **Procesamiento**:
   - Recorre cada oración en la lista `oraciones`.
   - Detecta entidades y aplica la fusión.
   - Imprime cada entidad con su tipo y score.

### 📦 Dependencias
- `transformers` de Hugging Face.

### ⚙️ Variable requerida
- `oraciones`: lista de textos u oraciones a procesar.


In [ ]:
from transformers import AutoTokenizer, pipeline

model_id = "FernandoValencia/xlm-roberta-base-finetuned-ner-pulmon"
tok = AutoTokenizer.from_pretrained(model_id, add_prefix_space=True)
ner = pipeline(
    "ner",
    model=model_id,
    tokenizer=tok,
    aggregation_strategy="simple"   # ← añade esto
)

def merge_spans(text, spans):
    fused = []
    for s in spans:
        if fused and s["entity_group"] == fused[-1]["entity_group"] \
           and s["start"] == fused[-1]["end"]:           # pegados en el texto
            fused[-1]["word"]  += text[s["start"]:s["end"]]
            fused[-1]["end"]    = s["end"]
            fused[-1]["score"]  = max(fused[-1]["score"], s["score"])
        else:
            fused.append(s.copy())
    return fused

for i, sent in enumerate(oraciones, 1):
    ents = merge_spans(sent, ner(sent))
    print(f"\nOración {i}:")
    for e in ents:
        print(f" - {e['word']} ({e['entity_group']}, {e['score']:.2f})")


Device set to use cpu



Oración 1:
 - fue (OCURRENCE_EVENT, 0.50)
 - diagnosticado (OCURRENCE_EVENT, 0.99)
 - adenocarcinoma de pulmón (CANCER_CONCEPT, 1.00)
 - estadio IV (STAGE, 1.00)
 - 12 de marzo de 2022 (DATE, 1.00)
 - metástasis en cerebro y hueso (CANCER_CONCEPT, 0.97)
 - T3N2M1 (TNM, 1.00)

Oración 2:
 - 5 de enero (DATE, 1.00)
 - quimioterapia (CHEMOTHERAPY, 0.99)
 - carboplatino (DRUG, 1.00)
 - pemetrexed (DRUG, 1.00)
 - 4 (QUANTITY, 1.00)
 - ciclos (METRIC, 1.00)
 - cada 21 días (FREQ, 0.99)

Oración 3:
 - cáncer de pulmón (CANCER_CONCEPT, 1.00)
 - padre (FAMILY, 0.65)
 - fumador (SMOKER_STATUS, 0.78)
 - 30 (QUANTITY, 0.99)
 - años (METRIC, 0.99)
 - inició tratamiento (OCURRENCE_EVENT, 0.98)
 - oterapia (RADIOTHERAPY, 0.37)
 - marzo (DATE, 0.99)

Oración 4:
 - 15 de junio (DATE, 1.00)

Oración 5:
 - 20 de abril (DATE, 1.00)
 - quimioterapia (CHEMOTHERAPY, 0.99)
 - radioterapia (RADIOTHERAPY, 0.87)

Oración 6:
 - pica (SURGERY, 0.50)
 - noviembre de 2021 (DATE, 1.00)
 - quimioterapia (CHEMOTHERAPY